# Credit Card Fraud Classifier
Detecting fraudulent credit card transactions using machine learning. It starts by loading and preprocessing a dataset containing transaction details and labels indicating fraudulence.

Upon preprocessing, the neural network is built layer by layer and is then trained on the training data set. After being optimized, with performance metrics evaluated on a separate testing dataset.

## Import Libraries

In [7]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from google.colab import drive
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import IsolationForest
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Dense, Dropout
from xgboost import XGBClassifier

## Load Dataset & Preprocess

In [2]:
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/ML/fraud-detection/card_transdata.csv'

Mounted at /content/drive


In [3]:
# Load dataset
data = pd.read_csv(file_path)

In [4]:
# Preprocess data (e.g., handle missing values, encode categorical variables, scale features)
X = data.drop('fraud', axis=1)  # Features
y = data['fraud']  # Labels

# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


## Sequential Neural Network

### Building / Training

In [16]:
# Standardization
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Building Neural Net
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# Compilation / Training
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train_scaled, y_train, epochs=10, batch_size=32, validation_split=0.2)


Epoch 1/10
20000/20000 [==============================] - 65s 3ms/step - loss: 0.0357 - accuracy: 0.9867 - val_loss: 0.0103 - val_accuracy: 0.9967
Epoch 2/10
20000/20000 [==============================] - 57s 3ms/step - loss: 0.0206 - accuracy: 0.9928 - val_loss: 0.0096 - val_accuracy: 0.9966
Epoch 3/10
20000/20000 [==============================] - 59s 3ms/step - loss: 0.0185 - accuracy: 0.9938 - val_loss: 0.0081 - val_accuracy: 0.9971
Epoch 4/10
20000/20000 [==============================] - 58s 3ms/step - loss: 0.0179 - accuracy: 0.9941 - val_loss: 0.0082 - val_accuracy: 0.9969
Epoch 5/10
20000/20000 [==============================] - 62s 3ms/step - loss: 0.0180 - accuracy: 0.9943 - val_loss: 0.0084 - val_accuracy: 0.9967
Epoch 6/10
20000/20000 [==============================] - 58s 3ms/step - loss: 0.0160 - accuracy: 0.9946 - val_loss: 0.0090 - val_accuracy: 0.9969
Epoch 7/10
20000/20000 [==============================] - 60s 3ms/step - loss: 0.0162 - accuracy: 0.9948 - val_loss: 0

### Evaluating Model

In [17]:
loss, accuracy = model.evaluate(X_test_scaled, y_test)
print(f'Test Loss: {loss}, Test Accuracy: {accuracy}')

6250/6250 [==============================] - 12s 2ms/step - loss: 0.0060 - accuracy: 0.9980
Test Loss: 0.006042294204235077, Test Accuracy: 0.998045027256012
6250/6250 [==============================] - 11s 2ms/step


## Random Forest

### Building / Training

In [5]:
rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

### Evaluating Model

In [6]:
y_pred = rf_classifier.predict(X_test)

# Evaluate model
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    182557
         1.0       1.00      1.00      1.00     17443

    accuracy                           1.00    200000
   macro avg       1.00      1.00      1.00    200000
weighted avg       1.00      1.00      1.00    200000

[[182557      0]
 [     2  17441]]


## Gradient Boosting

### Building / Training

In [8]:
# Train XGBoost classifier
xgb_classifier = XGBClassifier(n_estimators=100, random_state=42)
xgb_classifier.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=42, ...)

### Evaluating Model

In [9]:
y_pred = xgb_classifier.predict(X_test)

# Evaluate model
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00    182557
         1.0       0.99      0.99      0.99     17443

    accuracy                           1.00    200000
   macro avg       0.99      0.99      0.99    200000
weighted avg       1.00      1.00      1.00    200000

[[182367    190]
 [   179  17264]]


## Isolation Forest

### Building / Training

In [10]:
isolation_forest = IsolationForest(random_state=42)
isolation_forest.fit(X_train)

IsolationForest(random_state=42)

### Evaluating Model

In [11]:
y_pred = isolation_forest.predict(X_test)
# # Convert predictions to binary labels (1 for normal, -1 for anomaly)
# y_pred_binary = np.where(y_pred == 1, 0, 1)
# Evaluate model
y_pred_binary = y_pred
print(classification_report(y_test, y_pred_binary))
print(confusion_matrix(y_test, y_pred_binary))

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.

              precision    recall  f1-score   support

        -1.0       0.00      0.00      0.00         0
         0.0       0.00      0.00      0.00    182557
         1.0       0.07      0.58      0.12     17443

    accuracy                           0.05    200000
   macro avg       0.02      0.19      0.04    200000
weighted avg       0.01      0.05      0.01    200000

[[     0      0      0]
 [ 42150      0 140407]
 [  7396      0  10047]]
